In [1]:
import os
import sys
sys.path.insert(0, '../../')

In [2]:
import gym
import numpy as np

Mujoco wrappers test

In [3]:
from synchronized_wrapper.mujoco_wrapper import mujoco_env

In [4]:
game_id = 'HumanoidStandup-v2'
stack = 5
env = mujoco_env(game_id, stack=stack)
gym_env = gym.make(game_id)

print('action_space')
print(env.action_space.high[0], env.action_space.low[0], env.action_space.shape)
print(gym_env.action_space.high[0], gym_env.action_space.low[0], gym_env.action_space.shape) 

print('observation_space')
print(env.observation_space.high[0], env.observation_space.low[0], env.observation_space.shape)
print(gym_env.observation_space.high[0], gym_env.observation_space.low[0], gym_env.observation_space.shape)
assert env.observation_space.shape[0] == stack * gym_env.observation_space.shape[0]

assert env.observation_space.contains(env.reset())
while True:
    s, r, d, _ = env.step(np.random.uniform(low=-1.0, high=1.0, size=env.action_space.shape[0]))
    assert env.observation_space.contains(s)
    if d:
        break

env.close()
gym_env.close()

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
action_space
1.0 -1.0 (17,)
0.4 -0.4 (17,)
observation_space
inf -inf (1880,)
inf -inf (376,)


Atari wrappers test (just test for RewardClipEnv)

In [5]:
from synchronized_wrapper.atari_wrapper import atari_env

In [6]:
game_id = 'BeamRiderNoFrameskip-v4'
env = atari_env(game_id)

allowed_actions = list(range(env.action_space.n))

env.reset()
reward = 0
real_reward = 0
while True:
    s, r, d, info = env.step(np.random.choice(allowed_actions))
    
    assert 'was_real_done' in info
    assert 'real_reward' in info
    assert r in [-1.0, 0.0, 1.0]
    
    reward += r
    real_reward += info['real_reward']
    if d:
        print(info)
        if info['was_real_done']:
            print(reward, real_reward)
            break

env.close()

{'real_reward': 0.0, 'was_real_done': False, 'ale.lives': 2}
{'real_reward': 0.0, 'was_real_done': False, 'ale.lives': 1}
{'real_reward': 0.0, 'was_real_done': True, 'ale.lives': 0}
5.0 220.0


Agent test

In [7]:
from synchronized_wrapper.atari_wrapper import get_atari_env_fn
from synchronized_wrapper.mujoco_wrapper import get_mujoco_env_fn
from synchronized_wrapper.agent import Agent

In [8]:
env_fn = get_atari_env_fn('BeamRiderNoFrameskip-v4')
num_agents = 8

env = Agent(num_agents=num_agents, env_fn=env_fn, basename='test')

print('action_space')
print(env.action_space.nvec.dtype, type(env.action_space))

print('observation_space')
assert env.observation_space.shape == (num_agents, 84, 84, 4)
print(env.observation_space.high[0,0,0,0], env.observation_space.low[0,0,0,0], env.observation_space.shape, env.observation_space.dtype)

assert env.reset().shape == (num_agents, 84, 84, 4)
s, r, d, info = env.step(env.action_space.sample())
assert s.shape == (num_agents, 84, 84, 4)
assert r.shape == (num_agents,)
assert d.shape == (num_agents,) and d.dtype == np.bool
print(info)

env.close()

env = Agent(num_agents=num_agents, env_fn=env_fn, basename='test')
env.reset()

reward = np.zeros(num_agents)
for i in range(1000):
    s, r, d, info = env.step(env.action_space.sample())
    reward += r
    if d.any():
        print(reward[d])
        print(info)
        reward[d] = 0.0

env.close()

Process STDOUT and STDERR is being redirected to /tmp/raylogs/.
Waiting for redis server at 127.0.0.1:36039 to respond...
Waiting for redis server at 127.0.0.1:49320 to respond...
Starting local scheduler with the following resources: {'CPU': 32, 'GPU': 4}.

View the web UI at http://localhost:8891/notebooks/ray_ui92588.ipynb?token=20fa1e365010bb53be5ba23ee6dc093e3bf09af8d660de2f



All subagents are ready! 
action_space
int32 <class 'gym.spaces.multi_discrete.MultiDiscrete'>
observation_space
1.0 0.0 (8, 84, 84, 4) float32
{}
All subagents are ready! 
[7.]
{b'SubAgent-5': {b'length': 458, b'reward': 7.0}}
[6.]
{b'SubAgent-0': {b'length': 541, b'reward': 6.0}}
[7.]
{b'SubAgent-7': {b'length': 556, b'reward': 7.0}}
[5.]
{b'SubAgent-6': {b'length': 574, b'reward': 5.0}}
[5.]
{b'SubAgent-4': {b'length': 580, b'reward': 5.0}}
[1.]
{b'SubAgent-5': {b'length': 230, b'reward': 1.0}}
[8. 0.]
{b'SubAgent-0': {b'length': 205, b'reward': 0.0}, b'SubAgent-1': {b'length': 746, b'reward': 8.0}}
[2.]
{b'SubAgent-4': {b'length': 199, b'reward': 2.0}}
[2.]
{b'SubAgent-6': {b'length': 236, b'reward': 2.0}}
[5.]
{b'SubAgent-2': {b'length': 862, b'reward': 5.0}}
[4.]
{b'SubAgent-7': {b'length': 378, b'reward': 4.0}}
[1.]
{b'SubAgent-0': {b'real_reward': 308.0, b'real_length': 963, b'length': 217, b'reward': 1.0}}


In [9]:
from synchronized_wrapper.atari_wrapper import get_eval_atari_env_fn
from synchronized_wrapper.atari_wrapper import RewardClipEnv

fn = get_eval_atari_env_fn('BeamRiderNoFrameskip-v4')
env_fn = lambda :RewardClipEnv(fn())

env = Agent(num_agents=num_agents, env_fn=env_fn, basename='test')
env.reset()

reward = np.zeros(num_agents)
for i in range(2000):
    s, r, d, info = env.step(env.action_space.sample())
    reward += r
    if d.any():
        print(reward[d])
        print(info)
        reward[d] = 0.0

env.close()

All subagents are ready! 
[3.]
{b'SubAgent-2': {b'real_reward': 132.0, b'length': 854, b'reward': 3.0}}
[3.]
{b'SubAgent-0': {b'real_reward': 132.0, b'length': 1009, b'reward': 3.0}}
[3.]
{b'SubAgent-6': {b'real_reward': 132.0, b'length': 1051, b'reward': 3.0}}
[4.]
{b'SubAgent-5': {b'real_reward': 176.0, b'length': 1074, b'reward': 4.0}}
[8.]
{b'SubAgent-4': {b'real_reward': 352.0, b'length': 1332, b'reward': 8.0}}
[7.]
{b'SubAgent-7': {b'real_reward': 308.0, b'length': 1389, b'reward': 7.0}}
[11.]
{b'SubAgent-3': {b'real_reward': 484.0, b'length': 1565, b'reward': 11.0}}
[9.]
{b'SubAgent-1': {b'real_reward': 396.0, b'length': 1740, b'reward': 9.0}}
[4.]
{b'SubAgent-6': {b'real_reward': 176.0, b'length': 898, b'reward': 4.0}}


In [10]:
num_agents = 8
game_id = 'HumanoidStandup-v2'
env_fn = get_mujoco_env_fn(game_id)

from synchronized_wrapper.mujoco_wrapper import mujoco_env
gym_env = mujoco_env(game_id, stack=4)
env = Agent(num_agents=num_agents, env_fn=env_fn, basename='test')

print('action_space')
print(env.action_space.high[0][0], env.action_space.low[0][0], env.action_space.shape)
assert (env.action_space.high[0] == gym_env.action_space.high).all()
assert (env.action_space.low[0] == gym_env.action_space.low).all()
assert env.action_space.shape == (num_agents, *gym_env.action_space.shape)

print('observation_space')
print(env.observation_space.high[0][0], env.observation_space.low[0][0], env.observation_space.shape)
assert (env.observation_space.high[0] == gym_env.observation_space.high).all()
assert (env.observation_space.low[0] == gym_env.observation_space.low).all()
assert env.observation_space.shape == (num_agents, *gym_env.observation_space.shape)

gym_env.close()

assert env.observation_space.contains(env.reset())
s, r, d, info = env.step(env.action_space.sample())
assert env.observation_space.contains(s)
assert r.shape == (num_agents,)
assert d.shape == (num_agents,) and d.dtype == np.bool
print(info)

env.close()

env = Agent(num_agents=num_agents, env_fn=env_fn, basename='test')
env.reset()

reward = np.zeros(num_agents)
for i in range(1000):
    s, r, d, info = env.step(env.action_space.sample())
    reward += r
    if d.any():
        print(reward[d])
        print(info)
        reward[d] = 0.0

env.close()

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
All subagents are ready! 
action_space
1.0 -1.0 (8, 17)
observation_space
inf -inf (8, 1504)
{}
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
All subagents are ready! 
[33197.5725523  32923.98087056 31035.49105128 34029.08147479
 33604.92008781 33772.64100539 35806.21236894 32779.54982335]
{b'SubAgent-2': {b'length': 1000, b'reward': 33197.572552298014}, b'SubAgent-6': {b'length': 1000, b'reward': 33772.641005386075}, b'SubAgent-3': {b'length': 100

multiprocessing test

In [14]:
num_agents = 8
game_id = 'HumanoidStandup-v2'
env_fn = get_mujoco_env_fn(game_id)

env = Agent(num_agents=num_agents, env_fn=env_fn, basename='test', backend='multiprocessing')

env.reset()
reward = np.zeros(num_agents)
for i in range(1000):
    s, r, d, info = env.step(env.action_space.sample())
    reward += r
    if d.any():
        print(reward[d])
        print(info)
        reward[d] = 0.0

env.close()

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
All subagents are ready! 
[36885.07160586 34888.6535089  32893.8788857  32548.53548361
 31493.12474715 30692.2186922  32721.54136934 32904.92866246]
{b'SubAgent-2': {b'length': 1000, b'reward': 32548.53548360885}, b'SubAgent-6': {b'length': 1000, b'reward': 32721.54136934481}, b'SubAgent-5': {b'length': 1000, b'reward': 30692.218692201062}, b'SubAgent-4': {b'length': 1000, b'reward': 31493.12474714508}, b'SubAgent-3': {b'length': 1000, b'reward': 34888.653508895986}, b'SubAgent-0': {b'length': 1000, b'reward': 32893.87888569827}, b'SubAgent-7': {b'length': 1000, b'reward': 32904.92866246142}, b'SubAgent-1': {b'length': 1000, b'reward': 36885.07160585759}}
